In [10]:
!pip install cchardet
!pip install python-dotenv
!pip install selenium

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.



  Attempting uninstall: certifi
    Found existing installation: certifi 2020.12.5
    Uninstalling certifi-2020.12.5:
      Successfully uninstalled certifi-2020.12.5


In [22]:
# Para usar o sleep:
import time
# Para logs:
from tqdm import tqdm
import logging
import sys
sys.path.append('../utils')
from utils import get_logger, Clock, export_json
# Para web scraping:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import cchardet
# Para acessar as credenciais de login de um arquivo externo:
import os
import dotenv
# Para navegar na página além do botão 'Show more':
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
# Contador
from itertools import count
# Manipulação de JSON:
import json
#from dictdiffer import diff
# Para calcular similaridade entre strings:
#from thefuzz import fuzz

Inicializando o logger:

In [3]:
logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

### Obtendo os links das cervejas brasileiras do _Untappd_:

Fazendo uma requisição GET à página web que contém as cervejas brasileiras na seção _top rated_ da rede social _Untappd_ e transformando o resultado em um objeto BeautifulSoup:

In [4]:
base_url = 'https://untappd.com'
headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 OPR/97.0.0.0"
}
payload = {'country': 'brazil'}

page = requests.get(base_url + '/beer/top_rated', params=payload, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

#print(soup.prettify())

Minerando os links de todas as cervejas brasileiras e armazenando em uma lista:

In [5]:
logger.info("Coletando links de cervejas brasileiras do 'top rated'...")
brazilian_beer_links = [a_tag.get('href') for a_tag in soup.select('div.beer-item > a:first-child')]
logger.info("Feito.")
brazilian_beer_links

2023-10-01 12:26:29,391 - INFO - Coletando links de cervejas brasileiras do 'top rated'...
2023-10-01 12:26:29,410 - INFO - Feito.


['/b/spartacus-brewing-iii-three/4283069',
 '/b/spartacus-brewing-v-five/5285289',
 '/b/spartacus-brewing-iv-four/4756418',
 '/b/hocus-pocus-overdrive/1682105',
 '/b/spartacus-brewing-arete-tomo-2/4542383',
 '/b/salvador-brewing-co-cookie-de-chocolate/2908960',
 '/b/hocus-pocus-elephants-graveyard/2769185',
 '/b/cervejaria-dogma-rizoma/1536694',
 '/b/spartacus-brewing-arete-tomo-1/4098457',
 '/b/hocus-pocus-supersymmetry/1930404',
 '/b/spartacus-brewing-etera-tomo-5/4769967',
 '/b/koala-san-brew-cosmic-blur/3452440',
 '/b/trilha-cervejaria-pao-de-mel/3252352',
 '/b/hocus-pocus-rabbit-hole/2321900',
 '/b/spartacus-brewing-fwiw/3905760',
 '/b/cervejaria-dogma-mosaic-lover/1698497',
 '/b/salvador-brewing-co-engesa-oil-coffee-edition/3036359',
 '/b/spartacus-brewing-barrel-aged-icon-double-chocolate-peanut-butter/5076259',
 '/b/mad-dwarf-brewery-md-oloroso-2017/2957221',
 '/b/5-elementos-cervejaria-artesanal-coffee-and-pancake-brunch-stout/2546467',
 '/b/cervejaria-dogma-morning-gringo/265

In [6]:
number_of_brazilian_beer_links = len(brazilian_beer_links)
logger.info(f"Número total de cervejas no 'top rated' do Brasil: {number_of_brazilian_beer_links}")

2023-10-01 12:26:33,579 - INFO - Número total de cervejas no 'top rated' do Brasil: 50


### Acessando as páginas completas das cervejas com Selenium:

Carregando as credenciais de acesso:

In [7]:
dotenv.load_dotenv()
EMAIL = os.getenv('EMAIL')
PASSWORD = os.getenv('PASSWORD')

Definindo as opções do navegador que o Selenium usará:

In [8]:
options = Options()
options.add_argument("--disable-infobars")
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

Acessando a página completa de cada cerveja do _top rated_ do Brasil clicando no link 'Show more' e minerando os IDs de usuários:

In [17]:
browser = webdriver.Chrome(options=options)

browser.get(base_url + '/login')
browser.find_element(By.NAME,"username").send_keys(EMAIL)
browser.find_element(By.NAME,"password").send_keys(PASSWORD)
time.sleep(30)

top_rated_beers = []
logger.info("Iniciando uma sessão...")
with requests.Session() as session:
    logger.info("A iteração começará agora...")
    iteration_time = Clock("Coleta das infos das cervejas")
    with tqdm(total=number_of_brazilian_beer_links) as progress_bar:
        progress_bar.set_description("Coletando as top 50 cervejas melhor avaliadas do Brasil")
        for beer_url in brazilian_beer_links:
                try:
                    page = requests.get(base_url + beer_url, headers=headers)
                    soup = BeautifulSoup(page.text, 'html.parser')
                    name = soup.select('div.name > h1')[0].get_text()
                    brand = soup.select('p.brewery > a')[0].get_text()
                    style = soup.select('p.style')[0].get_text()
                    new_beer = {'name': name, 'style': style, 'brand': brand}
                    top_rated_beers.append(new_beer)
                    progress_bar.update(1)
                except IndexError:
                    print(f"Elemento não encontrado para a cerveja: {beer_url}")
                    progress_bar.update(1)
                    continue
        iteration_time.stop_watch()
    logger.info(f"Operação concluída! A lista filtrada possui {len(top_rated_beers)} cervejas.")
    
    print(top_rated_beers)

2023-10-01 12:59:46,548 - INFO - Iniciando uma sessão...
2023-10-01 12:59:46,551 - INFO - A iteração começará agora...
Coletando as top 50 cervejas melhor avaliadas do Brasil: 100%|█████████████████████████| 50/50 [00:49<00:00,  1.02it/s]2023-10-01 13:00:35,824 - INFO - Coleta das infos das cervejas levou 49s para ser executado.
Coletando as top 50 cervejas melhor avaliadas do Brasil: 100%|█████████████████████████| 50/50 [00:49<00:00,  1.01it/s]
2023-10-01 13:00:35,828 - INFO - Operação concluída! A lista filtrada possui 50 cervejas.


[{'name': 'III (Three)', 'style': 'Stout - Imperial / Double', 'brand': 'Spartacus Brewing'}, {'name': 'V (Five)', 'style': 'Stout - Imperial / Double', 'brand': 'Spartacus Brewing'}, {'name': 'IV (Four)', 'style': 'Stout - Imperial / Double', 'brand': 'Spartacus Brewing'}, {'name': 'Overdrive', 'style': 'IPA - Imperial / Double New England / Hazy', 'brand': 'Hocus Pocus'}, {'name': 'Aretê - Tomo 2', 'style': 'Stout - Imperial / Double', 'brand': 'Spartacus Brewing'}, {'name': 'Cookie de Chocolate', 'style': 'Stout - Imperial / Double Pastry', 'brand': 'Salvador Brewing Co.'}, {'name': "Elephants' Graveyard", 'style': 'Stout - Imperial / Double Pastry', 'brand': 'Hocus Pocus'}, {'name': 'Rizoma', 'style': 'IPA - Imperial / Double', 'brand': 'Cervejaria Dogma'}, {'name': 'Aretê - Tomo 1', 'style': 'Stout - Imperial / Double', 'brand': 'Spartacus Brewing'}, {'name': 'SuperSymmetry', 'style': 'IPA - Triple New England / Hazy', 'brand': 'Hocus Pocus'}, {'name': 'Etera - Tomo 5', 'style': '

In [26]:
path = '../data/'
file_name = 'top_rated_beers.json'

logger.info('Exportando a lista de cervejas coletadas na forma de um JSON...')
export_json(top_rated_beers, path + file_name)
logger.info('Exportado!')

2023-10-01 13:35:34,908 - INFO - Exportando a lista de cervejas coletadas na forma de um JSON...
2023-10-01 13:35:34,911 - INFO - Exportado!
